In [1]:
from profiler.core import *

In [2]:
pf = Profiler(workers=2)

In [3]:
pf.session.load_data(name='hospital', src='file', fpath='data/hospital.csv', embedtxt=False)

INFO:profiler.utility:[0.007762908935546875] Load Data start

INFO:profiler.dataset:inferred types of attributes: {
    "ProviderNumber": "text",
    "HospitalName": "text",
    "Address1": "text",
    "Address2": "categorical",
    "Address3": "categorical",
    "City": "text",
    "State": "categorical",
    "ZipCode": "text",
    "CountyName": "text",
    "PhoneNumber": "text",
    "HospitalType": "text",
    "HospitalOwner": "text",
    "EmergencyService": "categorical",
    "Condition": "text",
    "MeasureCode": "text",
    "MeasureName": "text",
    "Score": "text",
    "Sample": "text",
    "Stateavg": "text"
}
INFO:profiler.dataset:(possible types: numeric, categorical, text)
INFO:profiler.dataset:inferred operators of attributes: {'ProviderNumber': ['equal'], 'HospitalName': ['equal'], 'Address1': ['equal'], 'Address2': ['equal'], 'Address3': ['equal'], 'City': ['equal'], 'State': ['equal'], 'ZipCode': ['equal'], 'CountyName': ['equal'], 'PhoneNumber': ['equal'], 'HospitalTyp

In [4]:
pf.session.ds.change_dtypes(['ProviderNumber', 'ZipCode', 'PhoneNumber', 'State'], 
                         ['categorical', 'numeric', 'categorical', 'text'])

INFO:profiler.dataset:updated types of ProviderNumber to 'categorical'
INFO:profiler.dataset:updated types of ZipCode to 'numeric'
INFO:profiler.dataset:updated types of PhoneNumber to 'categorical'
INFO:profiler.dataset:updated types of State to 'text'
INFO:profiler.dataset:inferred operators of attributes: {'ProviderNumber': ['equal'], 'HospitalName': ['equal'], 'Address1': ['equal'], 'Address2': ['equal'], 'Address3': ['equal'], 'City': ['equal'], 'State': ['equal'], 'ZipCode': ['equal', 'greater_than', 'less_than'], 'CountyName': ['equal'], 'PhoneNumber': ['equal'], 'HospitalType': ['equal'], 'HospitalOwner': ['equal'], 'EmergencyService': ['equal'], 'Condition': ['equal'], 'MeasureCode': ['equal'], 'MeasureName': ['equal'], 'Score': ['equal'], 'Sample': ['equal'], 'Stateavg': ['equal']}
INFO:profiler.dataset:(possible operators: equal, notequal, greater_than, less_than)
INFO:profiler.dataset:updated inferred operators of attributes: {'ProviderNumber': ['equal'], 'HospitalName': ['

In [5]:
# TODO save embedding vectors
# pf.session.load_embedding()

In [6]:
pf.session.load_training_data()

INFO:profiler.utility:[0.12001991271972656] Create Training Data start

INFO:profiler.utility:[0.23513484001159668] Create Training Data execution time: 0.11511492729187012



In [ ]:
pf.session.learn_structure()

INFO:profiler.utility:[0.24075984954833984] Recover Moral Graph start

INFO:profiler.utility:[0.27988672256469727] Recover Moral Graph execution time: 0.03912687301635742

INFO:profiler.utility:[0.2805957794189453] Recover DAG start

INFO:graph:created node HospitalName_eq
INFO:graph:created node ProviderNumber_eq
INFO:graph:added directed edge HospitalName_eq -> ProviderNumber_eq
INFO:graph:created node Address1_eq
INFO:graph:added directed edge ProviderNumber_eq -> Address1_eq
INFO:graph:added directed edge HospitalName_eq -> Address1_eq
INFO:graph:created node City_eq
INFO:graph:added directed edge ProviderNumber_eq -> City_eq
INFO:graph:added directed edge HospitalName_eq -> City_eq
INFO:graph:added directed edge Address1_eq -> City_eq
INFO:graph:created node ZipCode_eq
INFO:graph:created node State_eq
INFO:graph:added directed edge ZipCode_eq -> State_eq
INFO:graph:created node CountyName_eq
INFO:graph:added directed edge ProviderNumber_eq -> CountyName_eq
INFO:graph:added directe

In [ ]:
s = pf.session.struct_engine

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure()
snsplt = sns.heatmap(s.inv_cov, cmap=sns.color_palette("RdBu_r", 1000), center=0)

In [ ]:
s.decomposed[1].print_edges()

In [ ]:
s.decomposed

In [ ]:
s.G.print_edges()

In [ ]:
s.width

In [ ]:
s.ordering